# D-Wave Examples

This notebook shows how to formulate and solve QUBO problems using D-Wave's dimod software package. The problems and formulations are either from the paper *Ising formulations of many NP problems* by Andrew Lucas (2014), or from *A Tutorial on Formulating and Using QUBO Models*, by Fred Glover, Gary Kochenberger, and Yu Du (2019).

## 1. Common functions

In [1]:
import dimod
import time
import numpy as np
from dwave.system import DWaveSampler, AutoEmbeddingComposite

def solve_qubo(Q,
               sampler="CPU", # CPU or QPU
               k=10,
               chain_strength=None):
    """
    Given an upper triangular matrix Q of size NxN, solves the quadratic unconstrained binary
    optimization (QUBO) problem given by
    
        minimize sum(x[i] * Q[i,j] * x[j]
                     for i in range(N),
                     for j in range(i+1, N))
    
    Uses dimod.SimulatedAnnealingSampler, which solves the problem k times through simulated
    annealing (on a regular CPU). This method returns the best solution found.
    """
    assert isinstance(Q, np.ndarray)
    assert sampler in ["CPU", "QPU"]
    n = Q.shape[0]
    nz = len(Q[Q!=0])
    print("Solving QUBO problem (%d vars, %d nz) on %s..." % (n, nz, sampler))
    
    start = time.time()
    if sampler == "CPU":
        sampler = dimod.SimulatedAnnealingSampler()
        response = sampler.sample_qubo(Q, num_reads=k)
    else:
        if chain_strength is None:
            chain_strength = int(10 * np.max(np.abs(Q)))
        sampler = AutoEmbeddingComposite(DWaveSampler(solver=dict(qpu=True)))
        response = sampler.sample_qubo(Q, num_reads=k, chain_strength=chain_strength)
    elapsed = time.time() - start
    
    print("Solved in %.2f seconds" % elapsed)
    solution = min(response.data(["sample", "energy"]), key=lambda s: s.energy)
    return solution, response

## 2. Number partitioning problem

Given a set $S=\left\{s_1,\ldots,s_n\right\}$ of $n$ positive integers, find a partition $(S_0, S_1)$ of $S$ minimizing the difference between $\sum_{s \in S_0} s$ and $\sum_{s \in S_1} s$.
In this formulation, the upper triangular matrix $Q$ is given by
$$
    Q_{i,j} = \begin{cases}
        s_i (s_j - c) & \text{if } i=j \\
        2 s_i s_j & \text{if } i>j,
    \end{cases}
$$
where $c = \sum_{s \in S} s$.

### 2.1 Create random instance and Q matrix

In [8]:
from scipy.stats import randint

# Pick random integer numbers between 1 and 100
np.random.seed(42)
S = randint(low=1, high=11).rvs(30)

# Generate Q matrix
c, n = sum(S), len(S)
Q = np.zeros((n, n), dtype=int)
for i in range(n):
    Q[i,i] = S[i] * (S[i] - c)
    for j in range(i + 1, n):
        Q[i,j] = 2 * S[i] * S[j]

### 2.2 Solve on the CPU

In [9]:
# Solve problem
solution, response = solve_qubo(Q)

# Display solution
S0 = [S[i] for (i, xi) in solution.sample.items() if xi >  0.5]
S1 = [S[i] for (i, xi) in solution.sample.items() if xi <= 0.5]
print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

Solving QUBO problem (30 vars, 465 nz) on CPU...
Solved in 18.87 seconds
sum(S0)       86
sum(S1)       86


### 2.3 Solve on the QPU

In [10]:
# Solve problem
solution, qresponse = solve_qubo(Q, sampler="QPU")

# Display solution
S0 = [S[i] for (i, xi) in solution.sample.items() if xi >  0.5]
S1 = [S[i] for (i, xi) in solution.sample.items() if xi <= 0.5]
print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

Solving QUBO problem (30 vars, 465 nz) on QPU...
Solved in 17.30 seconds
sum(S0)       86
sum(S1)       86


### 2.4 Inspect QPU solution and embedding

In [11]:
import dwave.inspector
dwave.inspector.show(qresponse)

'http://127.0.0.1:18000/?problemId=72672b81-b943-4dc6-ade7-24b0adce7c5d'